In [1]:
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
    ax.tick_params(width=0.5)

def ROI_planeID(expt, label):
    signals = expt.imaging_dataset().signals()[label]['rois']
    nROIs = len(signals)
    planeID = []
    for i in xrange(nROIs):
        planeID.append(signals[i]['polygons'][0][0][2])

    return np.asarray(planeID).astype('int')

def resample_trace(fluorescence_trace, number_of_data_points):
    y = fluorescence_trace
    x = np.arange(0, len(y))
    f = interpolate.interp1d(x,y)
    xnew = np.linspace(x.min(), x.max(), num = number_of_data_points)
    ynew = f(xnew)
    return ynew

def get_immobility_frames(expt, immobility_cutoff = 0.2):
    return np.where(expt.velocity()[0] < immobility_cutoff)[0]

def fill_nan(A):
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

def interpHeatmap(heatmaps):
    for j in range(heatmaps.shape[2]):
        heatmap = heatmaps[:, :, j]
        heatmaps[:, :, j] = np.reshape(fill_nan(heatmap.flatten()), heatmap.shape)
    return heatmaps

def heatmap(expt, label, file='from_file', n_bins = 100):
    sigs = expt.imagingData(dFOverF=file, label=label)[:, :, 0]
    bd = expt.behaviorData(imageSync=True)
    abs_pos = ba.absolutePosition(expt.find('trial'), imageSync=True)
    n_laps = int(abs_pos[-1]) + 1
    n_cells = sigs.shape[0]
    running = expt.runningIntervals(returnBoolList=True)[0]
    heatmap = np.zeros((n_laps, n_bins, n_cells))
    for j in xrange(sigs.shape[0]):
        fluo = np.zeros((n_laps, n_bins))
        counts = np.zeros((n_laps, n_bins))
        for i in xrange(sigs.shape[1]):
            if not running[i]:
                continue
            if np.isnan(sigs[j, i]):
                continue
            lap = int(abs_pos[i])
            posbin = int(np.mod(abs_pos[i] * 100, n_bins))
            fluo[lap, posbin] += sigs[j, i]
            counts[lap, posbin] += 1
            heatmap[:, :, j] = fluo/counts
    return heatmap